In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import glob

from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.resnet import preprocess_input, ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Flatten , Dense
from tensorflow.python.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

In [ ]:
TRAIN_PATH  = "./database/train_test_split/train"
VALID_PATH  = "./database/train_test_split/validation"
TEST_PATH   = "./database/train_test_split/test"

In [ ]:
objTrainGenerator = ImageDataGenerator(preprocessing_function= preprocess_input,
                                       zoom_range= 0.2,
                                       horizontal_flip= True,
                                       shear_range= 0.2,

                                       )

objTrain = objTrainGenerator.flow_from_directory(directory= TRAIN_PATH,
                                                 target_size=(224,224))

In [ ]:
objValidGenerator = ImageDataGenerator(preprocessing_function= preprocess_input  )

objValid = objValidGenerator.flow_from_directory(directory= VALID_PATH,
                                                 target_size=(224,224))

In [ ]:
objTestGenerator = ImageDataGenerator(preprocessing_function= preprocess_input )

objTest = objTestGenerator.flow_from_directory(directory= TEST_PATH ,
                                               target_size=(224,224),
                                               shuffle= False)

In [ ]:
classifier = ResNet50( input_shape=(224,224,3), include_top= False)

for layer in classifier.layers:
    layer.trainable = False

In [ ]:
objLayers = Flatten()(classifier.output)
objLayers = Dense(units=3 , activation='sigmoid', name = 'predictions' )(objLayers)

objModel = Model(classifier.input, objLayers)
objModel.compile( optimizer= 'adam' , loss = 'categorical_crossentropy', metrics=['accuracy'])
objModel.summary()

In [ ]:
earlyStopping = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
modelCheckpoint = ModelCheckpoint(filepath="bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only= True)

In [ ]:
objFit = objModel.fit_generator(objTrain, steps_per_epoch= 10, epochs= 30, validation_data=objValid , validation_steps= 16, callbacks=[earlyStopping, modelCheckpoint])

In [ ]:
objModel = load_model("bestmodel.h5")

nr_Acurracy = objModel.evaluate_generator(generator=objTest)[1]

print(f"A acurácia do modelo selecionado foi de = {nr_Acurracy}%")